# JOBS EVSE Automation 2.0
_____________________________________________________
### Argonne National Laboratory, Energy Systems & Infrastructure Analysis

Project: JOBS EVSE automation
Descrption: Automating the JOBS EVSE excel codebase from VBA in Excel to Python 
            for greater efficiency and ease for internal use.

Yue Ke,
Akshata Tiwari

## Excel Sheet Setup
Reading in Excel files and setting up User-Input excel sheets that will contain employment outputs

In [61]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
import math


In [62]:
# Download and read the excel sheet

# **** CHANGE FILE PATH ****

input_file_path = '/Users/akshatatiwari/Downloads/Input.xlsx'
inputs = pd.read_excel(input_file_path)
inputs


,Geography,Charger Power,Charger Cost,Number of chargers per station,Number of stations,Number of years for analysis,[Average] sessions per month,[Average] amount of kWh dispensed per charge session,Retail dollars per session,Access fees,Advertising,yes/no to include meters,yes/no to include transformer costs,yes/no to include onsite storage costs,yes/no to include indirect effects,yes/no to include induced effects
0,AR,150.0,87800,4,10,10,152,25.2,0.4,0,0.37,No,Yes,Yes,Yes,Yes
1,USA-National,50.0,27900,3,10,10,76,25.2,0.0,0,0.00,No,No,Yes,No,No
2,USA-National,150.0,87800,4,10,10,152,25.2,0.4,0,0.37,No,Yes,Yes,Yes,Yes
3,IL,150.0,87800,4,30,10,152,25.2,0.4,0,0.37,No,Yes,Yes,Yes,Yes
4,DE,6.6,530,3,40,10,60,6.6,0.0,0,0.00,No,Yes,No,Yes,No
5,IA,10.9,4900,3,10,10,63,14.3,0.7,0,1.55,No,No,No,No,Yes
6,IA,6.6,530,3,10,10,60,6.6,0.0,0,0.00,Yes,No,No,No,Yes
7,Census Division 2-Middle Atlantic,150.0,87800,4,10,10,152,52.9,1520.0,0,1400.00,No,Yes,Yes,Yes,Yes
8,Census Division 2-Middle Atlantic,150.0,87800,4,5,5,152,52.9,1520.0,0,1400.00,Yes,Yes,Yes,Yes,Yes


In [63]:
# Download and read the excel sheet

# **** CHANGE FILE PATH ****

jobs_evse_multipliers_file_path = '/Users/akshatatiwari/Downloads/JOBS EVSE Automation 2.0 - Multipliers.xlsx'
tier_mult = pd.read_excel(jobs_evse_multipliers_file_path, sheet_name = "TierEmp Mult")
type1_mult = pd.read_excel(jobs_evse_multipliers_file_path, sheet_name = "Type1Emp Muilt")
type2_mult = pd.read_excel(jobs_evse_multipliers_file_path, sheet_name = "Type2Emp Mult")



In [64]:
# Download and read the excel sheet

# **** CHANGE FILE PATH ****

electricity_rate_file_path = '/Users/akshatatiwari/Downloads/electricity_rate.xlsx'
elec_rate = pd.read_excel(electricity_rate_file_path)
elec_rate

,rRegions,USA-National,AK,AL,AR,AZ,CA,CO,CT,DC,...,WY,Census Division 1-New England,Census Division 2-Middle Atlantic,Census Division 3-East North Central,Census Division 4-West North Central,Census Division 5-South Atlantic,Census Division 6-East South Central,Census Division 7-West South Central,Census Division 8-Mountain,Census Division 9-Pacific
0,$/kWh Residental,0.131539,0.225714,0.125772,0.104071,0.122735,0.204490,0.123588,0.227147,0.126352,...,0.11110,0.213128,0.163900,0.136681,0.120949,0.118345,0.113182,0.112299,0.117487,0.184888
1,$/kWh Commerical,0.105913,0.195755,0.115467,0.086123,0.101102,0.175316,0.102911,0.165831,0.118538,...,0.09653,0.158036,0.122195,0.102827,0.097070,0.090586,0.107374,0.078381,0.094252,0.159411
2,Base rate,0.648235,0.560000,0.620000,0.360000,0.570000,0.700000,0.700000,0.220000,0.040000,...,0.37000,$ 0.1221951,NaN,NaN,1.267143,0.373750,0.650000,0.975000,0.431250,0.492000


## Station Development Formulas
Formulas for all EVSE Components that have to do with Station Development
Calculating total values based on formulas and multipliers for each JOBS EVSE Component.

#### Equipment Components
Includes Cable Cooling, Charger, Conduit and Cables, Trenching and Boring Labor, On-site Electrical Storage, Safety & Traffic Control, Load Center/Panels, Transformers, Meters, Misc. (mounting hardware, etc.)

In [65]:
# Global variable values:
# Employment deflator:
empdef = 0.83594203
station_equip_expenses = []

In [66]:
# Calculating total value for Station Development Employment
# NAME:         Cable Cooling (physical component)
# DESCRIPTION:  Air conditioning, refrigeration and warm air heating equipment manufacturing

# Function that will calculate cable cooling values for station development calc for: Producer, Wholesale, Shipping Margin
cable_cooling_cost2008_list = []
count = 0
def cable_cooling(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0
        if inputs["Charger Power"][i] <= 50:
            cost2008 = 0 
        elif inputs["Charger Power"][i] > 50:
            cost2008 = 500 * 4  
        
        if (count == 1):
            cable_cooling_cost2008_list.append(cost2008)

        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)


# List of margin values:
cable_cooling_PMList = [0.717282811, 0.282717189, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_cable_cooling = []
ptype1_cable_cooling = []
ptype2_cable_cooling = []
# Wholesale Margin Value:
wtier_cable_cooling = []
wtype1_cable_cooling = []
wtype2_cable_cooling = []
# Shipping Margin Value:
stier_cable_cooling = []
stype1_cable_cooling = []
stype2_cable_cooling = []

returnval = cable_cooling(cable_cooling_PMList[0], ptier_cable_cooling, ptype1_cable_cooling, ptype2_cable_cooling, 333415)
cable_cooling(cable_cooling_PMList[1], wtier_cable_cooling, wtype1_cable_cooling, wtype2_cable_cooling, 420000)
cable_cooling(cable_cooling_PMList[2], stier_cable_cooling, stype1_cable_cooling, stype2_cable_cooling, 484000)

# Appending cable_cooling_cost2008 to total station_equip_expenses list
station_equip_expenses.append(cable_cooling_cost2008_list)

print(ptier_cable_cooling)
print(ptype1_cable_cooling)
print(ptype2_cable_cooling)
print(wtier_cable_cooling)
print(wtype1_cable_cooling)
print(wtype2_cable_cooling)
print(stier_cable_cooling)
print(stype1_cable_cooling)
print(stype2_cable_cooling)
print(cable_cooling_cost2008_list)


[0.003454919762214714, 0.0, 0.003305047775311864, 0.002882672848809667, 0.0, 0.0, 0.0, 0.002883474051439695, 0.002883474051439695]
[0.006311101929637618, 0.0, 0.007379841177493861, 0.005837412518839576, 0.0, 0.0, 0.0, 0.0054437106617129995, 0.0054437106617129995]
[0.008861349780278423, 0.0, 0.01343071373461711, 0.009341874709156334, 0.0, 0.0, 0.0, 0.008712647281698783, 0.008712647281698783]
[0.001783716729990565, 0.0, 0.0015867149500904533, 0.0014127522869298369, 0.0, 0.0, 0.0, 0.0014597525763361374, 0.0014597525763361374]
[0.002921906375397545, 0.0, 0.0033551087619662634, 0.00279357637217506, 0.0, 0.0, 0.0, 0.0028183442991321804, 0.0028183442991321804]
[0.004236780787789102, 0.0, 0.006282450846524244, 0.004605841806264668, 0.0, 0.0, 0.0, 0.004605605471083779, 0.004605605471083779]
[0.0003242345466047479, 0.0, 0.00031380281673092504, 0.0002737650861581206, 0.0, 0.0, 0.0, 0.00028183875289827456, 0.00028183875289827456]
[0.0005080755345296399, 0.0, 0.0005576903658942, 0.00046321052577954

In [67]:
# Calculating total value for Station Development Employment
# NAME:         Charger (physical component)
# DESCRIPTION:  Other industrial machinery manufacturing

# Function that will calculate charger values for station development calc for: Producer, Wholesale, Shipping Margin
charger_cost2008_list = []
count = 0
def charger(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0
        
        if inputs["Charger Power"][i] <= 6.6:
            if inputs["Number of chargers per station"][i] == 1:
                cost2008 = 200 
            else:
                cost2008 = 530
        elif 6.6 < inputs["Charger Power"][i] and inputs["Charger Power"][i] <= 19.2:
            if inputs["Number of chargers per station"][i] == 1:
                cost2008 = 900 
            else:
                cost2008 = 4900 
        elif inputs["Charger Power"][i] == 50:
            cost2008 = (27900) 
        elif inputs["Charger Power"][i] == 150:
            cost2008 = (87800)
        elif inputs["Charger Power"][i] == 350:
            cost2008 = (140000) 
        cost2008 = cost2008 * inputs["Number of chargers per station"][i]   
        
        if (count == 1):
            charger_cost2008_list.append(cost2008)

        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)


# List of margin values:
charger_PMList = [0.857113566, 0.142886434, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_charger = []
ptype1_charger = []
ptype2_charger = []
# Wholesale Margin Value:
wtier_charger = []
wtype1_charger = []
wtype2_charger = []
# Shipping Margin Value:
stier_charger = []
stype1_charger = []
stype2_charger = []

returnval = charger(charger_PMList[0], ptier_charger, ptype1_charger, ptype2_charger, "33329A")
charger(charger_PMList[1], wtier_charger, wtype1_charger, wtype2_charger, 420000)
charger(charger_PMList[2], stier_charger, stype1_charger, stype2_charger, 484000)

# Appending charger_cost2008 to total station_equip_expenses list
station_equip_expenses.append(charger_cost2008_list)

print(ptier_charger)
print(ptype1_charger)
print(ptype2_charger)
print(wtier_charger)
print(wtype1_charger)
print(wtype2_charger)
print(stier_charger)
print(stype1_charger)
print(stype2_charger)
print(charger_cost2008_list)

[0.717791005292855, 0.1737903969902237, 0.7292137087570676, 0.6831787556550997, 0.0029574851406385778, 0.03206051552230446, 0.0034677700462900753, 0.706417836244412, 0.706417836244412]
[1.34915997354845, 0.4102843692145201, 1.7215277236336852, 1.4510716770114318, 0.004194305426453632, 0.058128920693490224, 0.006287413870928535, 1.3888174660565138, 1.3888174660565138]
[1.9614605859841863, 0.7659773008145201, 3.2139931666195873, 2.3479953210199542, 0.005747988165568024, 0.08362917977865224, 0.009045605159731775, 2.236622185334237, 2.236622185334237]
[0.15830301300224414, 0.03356086606942532, 0.1408193090033712, 0.12538030276140713, 0.0004734041011778825, 0.006451598545865862, 0.0006978259651650832, 0.12955152978411613, 0.12955152978411613]
[0.25931616559897613, 0.07096445130379983, 0.2977624288876284, 0.24792701068040648, 0.0008124087780313643, 0.01041352521288209, 0.0011263608903729608, 0.25012513855419305, 0.25012513855419305]
[0.37600990833369824, 0.13288114001568355, 0.55756100804669

In [68]:
# Calculating total value for Station Development Employment
# NAME:         Conduit Cables (physical component)
# DESCRIPTION:  Communication and energy wire and cable manufacturing

# Function that will calculate Conduit Cables values for station development calc for: Producer, Wholesale, Shipping Margin
conduit_cables_cost2008_list = []
count = 0
def conduit_cables(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0
        
        if inputs["Charger Power"][i] <= 6.6:
            cost2008 = 7
        elif inputs["Charger Power"][i] > 6.6 and inputs["Charger Power"][i] <= 10.9:
            cost2008 = 20
        else:
            cost2008 = 25
       
        cost2008 = cost2008 * 75  

        if (count == 1):
            conduit_cables_cost2008_list.append(cost2008) 

        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)


# List of margin values:
conduit_cables_PMList = [0.793550332, 0.206449668, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_conduit_cables = []
ptype1_conduit_cables = []
ptype2_conduit_cables = []
# Wholesale Margin Value:
wtier_conduit_cables = []
wtype1_conduit_cables = []
wtype2_conduit_cables = []
# Shipping Margin Value:
stier_conduit_cables = []
stype1_conduit_cables = []
stype2_conduit_cables = []

returnval = conduit_cables(conduit_cables_PMList[0], ptier_conduit_cables, ptype1_conduit_cables, ptype2_conduit_cables, 335920)
conduit_cables(conduit_cables_PMList[1], wtier_conduit_cables, wtype1_conduit_cables, wtype2_conduit_cables, 420000)
conduit_cables(conduit_cables_PMList[2], stier_conduit_cables, stype1_conduit_cables, stype2_conduit_cables, 484000)

# Appending conduit_cables_cost2008 to total station_equip_expenses list
station_equip_expenses.append(conduit_cables_cost2008_list)

print(ptier_conduit_cables)
print(ptype1_conduit_cables)
print(ptype2_conduit_cables)
print(wtier_conduit_cables)
print(wtype1_conduit_cables)
print(wtype2_conduit_cables)
print(stier_conduit_cables)
print(stype1_conduit_cables)
print(stype2_conduit_cables)
print(conduit_cables_cost2008_list)

[0.0024894861180668982, 0.0029912338984530774, 0.0029912338984530774, 0.0025352382191232586, 0.0, 0.0033625179221381356, 0.0011768812727483476, 0.0034218900498371595, 0.0034218900498371595]
[0.006264293918891736, 0.008142437794979123, 0.008142437794979123, 0.0061172762989225105, 0.0, 0.005829261069818672, 0.0020402413744365355, 0.007072020165998458, 0.007072020165998458]
[0.009135988343468265, 0.01504194236123366, 0.01504194236123366, 0.009973441503580609, 0.0, 0.008016763849787155, 0.0028058673474255047, 0.010922712633534181, 0.010922712633534181]
[0.0012211222282386958, 0.0010862559412358902, 0.0010862559412358902, 0.0009671621012234384, 0.0002258485913425, 0.0009511838125043731, 0.00033291433437653055, 0.000999338229395981, 0.000999338229395981]
[0.0020003203220778074, 0.002296888187743289, 0.002296888187743289, 0.0019124663389592273, 0.00038757876760286427, 0.0015353057917633088, 0.000537357027117158, 0.0019294223194948209, 0.0019294223194948209]
[0.0029004757925723385, 0.004300929

In [69]:
# Calculating total value for Station Development Employment
# NAME:         Trenching and Boring Labor
# DESCRIPTION:  Nonresidential structures

# Function that will calculate Trenching and Boring Labor values for station development calc for: Producer, Wholesale Margin
trenching_cost2008_list = []
count = 0
def trenching(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0
        
        if inputs["Charger Power"][i] <= 6.6:
            cost2008 = 0
        else:
            cost2008 = 80
       
        cost2008 = cost2008 * 75 
        
        if (count == 1):
            trenching_cost2008_list.append(cost2008)
  
        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)

# List of margin values:
trenching_PMList = [1, 1]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_trenching = []
ptype1_trenching = []
ptype2_trenching = []
# Wholesale Margin Value:
wtier_trenching = []
wtype1_trenching = []
wtype2_trenching = []

trenching(trenching_PMList[0], ptier_trenching, ptype1_trenching, ptype2_trenching, "2332E0")
trenching(trenching_PMList[1], wtier_trenching, wtype1_trenching, wtype2_trenching, 420000)

# Appending trenching_cost2008 to total station_equip_expenses list
station_equip_expenses.append(trenching_cost2008_list)

print(ptier_trenching)
print(ptype1_trenching)
print(ptype2_trenching)
print(wtier_trenching)
print(wtype1_trenching)
print(wtype2_trenching)
print(trenching_cost2008_list)

[0.048238784304099426, 0.04372931605460144, 0.04372931605460144, 0.03823932603960259, 0.0, 0.04379196623125828, 0.0, 0.03934246930309576, 0.03934246930309576]
[0.059965632768426, 0.060070961464206, 0.060070961464206, 0.049649940929819995, 0.0, 0.054223212587544006, 0.0, 0.049461352407851995, 0.049461352407851995]
[0.08381706514519799, 0.10884216013209001, 0.10884216013209001, 0.07948304009646, 0.0, 0.07729822200685199, 0.0, 0.078686554530276, 0.078686554530276]
[0.018927572847265737, 0.016837125705403643, 0.016837125705403643, 0.014991153795001516, 0.0, 0.01842935998336162, 0.0, 0.015489888480068369, 0.015489888480068369]
[0.031005257081105998, 0.035602102304076004, 0.035602102304076004, 0.029643507514236, 0.0, 0.029746829949144, 0.0, 0.029906327688468, 0.029906327688468]
[0.04495779831543, 0.06666503938525199, 0.06666503938525199, 0.04887401953757401, 0.0, 0.043166708921952006, 0.0, 0.048871511711484, 0.048871511711484]
[6000, 6000, 6000, 6000, 0, 6000, 0, 6000, 6000]


In [70]:
# Calculating total value for Station Development Employment
# NAME:         On-site Electrical Storage
# DESCRIPTION:  Storage battery manufacturing

# Function that will calculate Electrical Storage values for station development calc for: Producer, Wholesale, Shipping Margin
electrical_storage_cost2008_list = []
count = 0
def electrical_storage(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0
        
        if inputs["Charger Power"][i] == 50:
            cost2008 = 83200
        elif inputs["Charger Power"][i] == 150:
            cost2008 = 332800
        elif inputs["Charger Power"][i] == 350:
            cost2008 = 388266.6667
        
        if (count == 1):
            if str(inputs["yes/no to include onsite storage costs"][i]).strip().lower() == "yes":
                electrical_storage_cost2008_list.append(cost2008) 
            else:
                electrical_storage_cost2008_list.append(0)
       
        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]
        
        if str(inputs["yes/no to include onsite storage costs"][i]).strip().lower() == "yes":
            tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
            type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
            type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        else:
            tierlist[i] = 0
            type1list[i] = 0
            type2list[i] = 0

# List of margin values:
electrical_storage_PMList = [0.709280709, 0.290719291, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_electrical_storage = []
ptype1_electrical_storage = []
ptype2_electrical_storage = []
# Wholesale Margin Value:
wtier_electrical_storage = []
wtype1_electrical_storage = []
wtype2_electrical_storage = []
# Shipping Margin Value:
stier_electrical_storage = []
stype1_electrical_storage = []
stype2_electrical_storage = []

returnval = electrical_storage(electrical_storage_PMList[0], ptier_electrical_storage, ptype1_electrical_storage, ptype2_electrical_storage, 335911)
electrical_storage(electrical_storage_PMList[1], wtier_electrical_storage, wtype1_electrical_storage, wtype2_electrical_storage, 420000)
electrical_storage(electrical_storage_PMList[2], stier_electrical_storage, stype1_electrical_storage, stype2_electrical_storage, 484000)

# Appending electrical_storage_cost2008 to total station_equip_expenses list
station_equip_expenses.append(electrical_storage_cost2008_list)

print(ptier_electrical_storage)
print(ptype1_electrical_storage)
print(ptype2_electrical_storage)
print(wtier_electrical_storage)
print(wtype1_electrical_storage)
print(wtype2_electrical_storage)
print(stier_electrical_storage)
print(stype1_electrical_storage)
print(stype2_electrical_storage)
print(electrical_storage_cost2008_list)


[0.5112869433774286, 0.12437229560897478, 0.4974891824358991, 0.4128539373632344, 0, 0, 0, 0.48350103913511083, 0.48350103913511083]
[0.7701515228094208, 0.25029924491306177, 1.001196979652247, 0.7033379676920061, 0, 0, 0, 0.8298811835715043, 0.8298811835715043]
[1.0975892468550148, 0.46389641879293053, 1.8555856751717221, 1.1510835020007062, 0, 0, 0, 1.3197354380871043, 1.3197354380871043]
[0.3052114656452407, 0.06787563116606579, 0.27150252466426317, 0.2417358030217062, 0, 0, 0, 0.24977801453111068, 0.24977801453111068]
[0.4999669018734688, 0.1435230221006461, 0.5740920884025844, 0.4780083768951219, 0, 0, 0, 0.4822464126552154, 0.4822464126552154]
[0.724954838465606, 0.2687472733860032, 1.0749890935440127, 0.7881048064328003, 0, 0, 0, 0.7880643671602804, 0.7880643671602804]
[0.05395262855503005, 0.013054197176006482, 0.05221678870402593, 0.04555451033671127, 0, 0, 0, 0.046897968482272887, 0.046897968482272887]
[0.0845437689457321, 0.023199919221198722, 0.09279967688479489, 0.07707823

In [71]:
# Calculating total value for Station Development Employment
# NAME:         Safety & Traffic Control
# DESCRIPTION:  Transportation structures and highways and streets

# Function that will calculate Safety & Traffic Control values for station development calc for: Producer, Wholesale, Shipping Margin
safety_traffic_cost2008_list = []
count = 0
def safety_traffic(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0
        if inputs["Number of chargers per station"][i] == 3 and inputs["Charger Power"][i] <= 19.2:
            cost2008 = 1000 
        else:
            cost2008 = 3000 
       
        
        if (count == 1):
            safety_traffic_cost2008_list.append(cost2008) 

        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)

# List of margin values:
safety_traffic_PMList = [1, 0, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_safety_traffic = []
ptype1_safety_traffic = []
ptype2_safety_traffic = []
# Wholesale Margin Value:
wtier_safety_traffic = []
wtype1_safety_traffic = []
wtype2_safety_traffic = []
# Shipping Margin Value:
stier_safety_traffic = []
stype1_safety_traffic = []
stype2_safety_traffic = []

returnval = safety_traffic(safety_traffic_PMList[0], ptier_safety_traffic, ptype1_safety_traffic, ptype2_safety_traffic, "2332F0")
safety_traffic(safety_traffic_PMList[1], wtier_safety_traffic, wtype1_safety_traffic, wtype2_safety_traffic, 420000)
safety_traffic(safety_traffic_PMList[2], stier_safety_traffic, stype1_safety_traffic, stype2_safety_traffic, 484000)

# Appending safety_traffic_cost2008 to total station_equip_expenses list
station_equip_expenses.append(safety_traffic_cost2008_list)

print(ptier_safety_traffic)
print(ptype1_safety_traffic)
print(ptype2_safety_traffic)
print(wtier_safety_traffic)
print(wtype1_safety_traffic)
print(wtype2_safety_traffic)
print(stier_safety_traffic)
print(stype1_safety_traffic)
print(stype2_safety_traffic)
print(safety_traffic_cost2008_list)



[0.013873686293117344, 0.012389527713641834, 0.012389527713641834, 0.010475743479269967, 0.0034595964751967844, 0.0041987224633527045, 0.0041987224633527045, 0.010884147417952163, 0.010884147417952163]
[0.020040539850408, 0.020789627503491, 0.020789627503491, 0.016344254976356998, 0.004474296121372, 0.005852179369421, 0.005852179369421, 0.015881059497534, 0.015881059497534]
[0.027998122816587003, 0.037959208046067006, 0.037959208046067006, 0.026494430293023, 0.006105804181323, 0.008321468531838, 0.008321468531838, 0.025468729422213, 0.025468729422213]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.00048635181990712184, 0.00047070422509638756, 0.00047070422509638756, 0.00041064762923718085, 0.00012716426215740224, 0.00014077908288039203, 0.00014077908288039203, 0.0004227581293474118, 0.0004227581293474118]
[0.00076211330179446, 0.0008365355488412999, 0.0008365355488412999, 0.00069481578866931, 

In [72]:
# Calculating total value for Station Development Employment
# NAME:         Load Center/Panels
# DESCRIPTION:  Switchgear and switchboard apparatus manufacturing

# Function that will calculate Safety & Traffic Control values for station development calc for: Producer, Wholesale, Shipping Margin
load_center_cost2008_list = []
count = 0
def load_center(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0

        if inputs["Charger Power"][i] <= 6.6:
            cost2008 = 5 
        elif 6.6 < inputs["Charger Power"][i] and inputs["Charger Power"][i] <= 19.2:
            cost2008 = 10 
        else:
            cost2008 = 40 

        if inputs["Number of chargers per station"][i] == 3:
            cost2008 = cost2008 * 6
        elif inputs["Number of chargers per station"][i] == 4 or inputs["Number of chargers per station"][i] == 2:
            cost2008 = cost2008 * 4
        
        
        if (count == 1):
            load_center_cost2008_list.append(cost2008) 

        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
   


# List of margin values:
load_center_PMList = [0.785214135, 0.214785865, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_load_center = []
ptype1_load_center = []
ptype2_load_center = []
# Wholesale Margin Value:
wtier_load_center = []
wtype1_load_center = []
wtype2_load_center = []
# Shipping Margin Value:
stier_load_center = []
stype1_load_center = []
stype2_load_center = []

returnval = load_center(load_center_PMList[0], ptier_load_center, ptype1_load_center, ptype2_load_center, 335313)
load_center(load_center_PMList[1], wtier_load_center, wtype1_load_center, wtype2_load_center, 420000)
load_center(load_center_PMList[2], stier_load_center, stype1_load_center, stype2_load_center, 484000)

# Appending load_center_cost2008 to total station_equip_expenses list
station_equip_expenses.append(load_center_cost2008_list)

print(ptier_load_center)
print(ptype1_load_center)
print(ptype2_load_center)
print(wtier_load_center)
print(wtype1_load_center)
print(wtype2_load_center)
print(stier_load_center)
print(stype1_load_center)
print(stype2_load_center)
print(load_center_cost2008_list)



[0.0002993340507676826, 0.0004813065893361714, 0.00032087105955744765, 0.00026753423744150563, 6.0433417894164764e-05, 0.00010555543010255985, 5.2777715051279926e-05, 0.00029532720326150557, 0.00029532720326150557]
[0.0005150641011559514, 0.0010131985012115747, 0.0006754656674743831, 0.0005295572695917163, 7.825523283115395e-05, 0.0001876353325503104, 9.38176662751552e-05, 0.0005403011183669244, 0.0005403011183669244]
[0.0007603977349671585, 0.0019069228834918011, 0.0012712819223278675, 0.000881510212069506, 0.00010524482068177789, 0.00028161250244547876, 0.00014080625122273938, 0.0008829910358009864, 0.0008829910358009864]
[0.0001084100028360129, 0.00014465506434995426, 9.643670956663617e-05, 8.586367827219821e-05, 1.3426748243417685e-05, 3.958366025422711e-05, 1.9791830127113555e-05, 8.872024255853385e-05, 8.872024255853385e-05]
[0.00017758642564567274, 0.00030587313356797826, 0.00020391542237865217, 0.00016978683741544474, 2.3041642660529087e-05, 6.389198601634799e-05, 3.19459930081

In [73]:
# Calculating total value for Station Development Employment
# NAME:         Transformers
# DESCRIPTION:  Power, distribution, and specialty transformer manufacturing

# Function that will calculate Transformers values for station development calc for: Producer, Wholesale, Shipping Margin
transformers_cost2008_list = []
count = 0

def transformers(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1
    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)

        # Find tier employment per charger:
        cost2008 = 0

        if inputs["Charger Power"][i] <= 6.6:
            cost2008 = 0
        elif 6.6 < inputs["Charger Power"][i] and inputs["Charger Power"][i] <= 10.9:
            cost2008 = 11219.51132
        elif inputs["Charger Power"][i] == 50:
            cost2008 = 16446.835
        elif inputs["Charger Power"][i] == 150:
            cost2008 = 37865.26
        elif inputs["Charger Power"][i] == 350:
            cost2008 = 42918.94
        
        if (count == 1):
            if str(inputs["yes/no to include transformer costs"][i]).strip().lower() == "yes":
                transformers_cost2008_list.append(cost2008) 
            else:
                transformers_cost2008_list.append(0)


        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]

        if str(inputs["yes/no to include transformer costs"][i]).strip().lower() == "yes":
            tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
            type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
            type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        else:
            tierlist[i] = 0
            type1list[i] = 0
            type2list[i] = 0
        

# List of margin values:
transformers_PMList = [0.79003035, 0.20996965, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_transformers = []
ptype1_transformers = []
ptype2_transformers = []
# Wholesale Margin Value:
wtier_transformers = []
wtype1_transformers = []
wtype2_transformers = []
# Shipping Margin Value:
stier_transformers = []
stype1_transformers = []
stype2_transformers = []

returnval = transformers(transformers_PMList[0], ptier_transformers, ptype1_transformers, ptype2_transformers, 335311)
transformers(transformers_PMList[1], wtier_transformers, wtype1_transformers, wtype2_transformers, 420000)
transformers(transformers_PMList[2], stier_transformers, stype1_transformers, stype2_transformers, 484000)

# Appending transformers_cost2008 to total station_equip_expenses list
station_equip_expenses.append(transformers_cost2008_list)

print(ptier_transformers)
print(ptype1_transformers)
print(ptype2_transformers)
print(wtier_transformers)
print(wtype1_transformers)
print(wtype2_transformers)
print(stier_transformers)
print(stype1_transformers)
print(stype2_transformers)
print(transformers_cost2008_list)


[0.09934816674637163, 0, 0.09446418797182643, 0.07367647633831624, 0.0, 0, 0, 0.09308859074236911, 0.09308859074236911]
[0.15352272207316806, 0, 0.1724254823049748, 0.13252187798972942, 0.0, 0, 0, 0.14686586961423573, 0.14686586961423573]
[0.21919599753390462, 0, 0.32145445386398, 0.22205679363195818, 0.0, 0, 0, 0.23321739938861297, 0.23321739938861297]
[0.025080786051385612, 0, 0.022310750086401984, 0.019864666432926486, 0.0, 0, 0, 0.02052553605596291, 0.02052553605596291]
[0.041084835630774766, 0, 0.04717608105769699, 0.03928039140446884, 0.0, 0, 0, 0.03962865246324759, 0.03962865246324759]
[0.05957324428174282, 0, 0.08833734802770754, 0.06476259990568267, 0.0, 0, 0, 0.06475927680397676, 0.06475927680397676]
[0.006138612704085448, 0, 0.005941112622124414, 0.005183093083149818, 0.0, 0, 0, 0.005335948828284459, 0.005335948828284459]
[0.009619206107301898, 0, 0.010558545352039508, 0.008769793496689493, 0.0, 0, 0, 0.00834969272649952, 0.00834969272649952]
[0.013095388045444809, 0, 0.0181

In [74]:
# Calculating total value for Station Development Employment
# NAME:         Meters
# DESCRIPTION:  Electrical Meters

# Function that will calculate Meters values for station development calc for: Producer, Wholesale, Shipping Margin
meters_cost2008_list = []
count = 0
def meters(marginval, tierlist, type1list, type2list, code):
    global count
    count = count + 1

    # Finding total Tier, Type I, Type II Employment for each run:
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)
       
        # Find tier employment per charger:
        cost2008 = 0

        if inputs["Charger Power"][i] <= 6.6:
            cost2008 = 0
        else:
            cost2008 = 2000
        
        if inputs["Charger Power"][i] == 150 or inputs["Charger Power"][i] == 350:
            cost2008 = cost2008 * 4
        else:
            cost2008 = cost2008 * 6
        
        if (count == 1):
            if str(inputs["yes/no to include meters"][i]).strip().lower() == "yes":
                meters_cost2008_list.append(cost2008) 
            else:
                meters_cost2008_list.append(0) 

        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]
        
        if str(inputs["yes/no to include meters"][i]).strip().lower() == "yes":
            tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
            type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
            type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        else:
            tierlist[i] = 0
            type1list[i] = 0
            type2list[i] = 0


# List of margin values:
meters_PMList = [0.878689182, 0.121310818, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_meters = []
ptype1_meters = []
ptype2_meters = []
# Wholesale Margin Value:
wtier_meters = []
wtype1_meters = []
wtype2_meters = []
# Shipping Margin Value:
stier_meters = []
stype1_meters = []
stype2_meters = []

returnval = meters(meters_PMList[0], ptier_meters, ptype1_meters, ptype2_meters, 334515)
meters(meters_PMList[1], wtier_meters, wtype1_meters, wtype2_meters, 420000)
meters(meters_PMList[2], stier_meters, stype1_meters, stype2_meters, 484000)

# Appending meters_cost2008 to total station_equip_expenses list
station_equip_expenses.append(meters_cost2008_list)

print(ptier_meters)
print(ptype1_meters)
print(ptype2_meters)
print(wtier_meters)
print(wtype1_meters)
print(wtype2_meters)
print(stier_meters)
print(stype1_meters)
print(stype2_meters)
print(meters_cost2008_list)



[0, 0, 0, 0, 0, 0, 0.0, 0, 0.017877339781053853]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.03288358312760407]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.05748163155007559]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.0025054547229944944]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.004837281433685471]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.007904857416822272]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.0011273550115930982]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.00176408512214088]
[0, 0, 0, 0, 0, 0, 0.0, 0, 0.00265570757887512]
[0, 0, 0, 0, 0, 0, 0, 0, 8000]


In [75]:
# Calculating total value for Station Development Employment
# NAME:         Misc. (mounting hardware, etc.)
# DESCRIPTION:  Other concrete product manufacturing

# Function that will calculate Transformers values for station development calc for: Producer, Wholesale, Shipping Margin
misc_cost2008_list = []
count = 0
def misc(marginval, tierlist, type1list, type2list, code):
    # Finding total Tier, Type I, Type II Employment for each run:
    global count
    count = count + 1
    for i in range(len(inputs)):
        tierlist.append(0)
        type1list.append(0)
        type2list.append(0)
        
        # Find tier employment per charger:
        cost2008 = 0

        if inputs["Number of chargers per station"][i] == 1:
            cost2008 = 100
        elif inputs["[Average] sessions per month"][i] == 114:
            cost2008 = 2000 * 3
        elif inputs["Charger Power"][i] <= 50:
            cost2008 = 1000 * 3
        elif inputs["Charger Power"][i] == 150:
            cost2008 = 2000 * 4
        elif inputs["Charger Power"][i] == 350:
            cost2008 = 2000 * 2

        if (count == 1):
            misc_cost2008_list.append(cost2008) 
        
        cost2008 = cost2008 * empdef 
        row_num = tier_mult[tier_mult['Code'] == code].index[0]
        tierlist[i] = ((cost2008 * marginval * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type1list[i] = ((cost2008 * marginval * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
        type2list[i] = ((cost2008 * marginval * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)


# List of margin values:
misc_PMList = [0.886582374, 0.113417626, 0.03]

# List for tier, type I, and type II Employment for each margin value for cable cooling:
# Producer Margin Value:
ptier_misc = []
ptype1_misc = []
ptype2_misc = []
# Wholesale Margin Value:
wtier_misc = []
wtype1_misc = []
wtype2_misc = []
# Shipping Margin Value:
stier_misc = []
stype1_misc = []
stype2_misc = []

misc(misc_PMList[0], ptier_misc, ptype1_misc, ptype2_misc, 327390)
misc(misc_PMList[1], wtier_misc, wtype1_misc, wtype2_misc, 420000)
misc(misc_PMList[2], stier_misc, stype1_misc, stype2_misc, 484000)

# Appending misc_cost2008 to total station_equip_expenses list
station_equip_expenses.append(misc_cost2008_list)

print(ptier_misc)
print(ptype1_misc)
print(ptype2_misc)
print(wtier_misc)
print(wtype1_misc)
print(wtype2_misc)
print(stier_misc)
print(stype1_misc)
print(stype2_misc)

print(misc_cost2008_list)



[0.031337388815087804, 0.010739087924665667, 0.028637567799108442, 0.024624765254933564, 0.009142325082480304, 0.011899372110211952, 0.011899372110211952, 0.02968182528623822, 0.02968182528623822]
[0.0492905788672516, 0.020554614287810084, 0.054812304767493554, 0.044255628116166595, 0.012461903319928903, 0.018121553786641785, 0.018121553786641785, 0.046683574810195465, 0.046683574810195465]
[0.06653937623542928, 0.035628783698228465, 0.09501008986194255, 0.06818172357180534, 0.016281250234766613, 0.024471568217178802, 0.024471568217178802, 0.06902483473149008, 0.06902483473149008]
[0.002862293837705254, 0.0009548134130852281, 0.0025461691015606085, 0.0022670147659066167, 0.0007089991283495791, 0.0010451071290061374, 0.0010451071290061374, 0.0023424351712188034, 0.0023424351712188034]
[0.004688723535544976, 0.002018952961968715, 0.005383874565249906, 0.004482794998103744, 0.0012167134041607128, 0.0016869074169288062, 0.0016869074169288062, 0.004522539585072144, 0.004522539585072144]
[0.

#### Non-Equipment Components
Includes: Equipment Installation, Site Prep & Construction, Electrical Infrastrastructure & Make Ready, Engineering & Design, Permitting,	Contingencies - Install, Contingencies - Site Prep & Construction, Contingencies - Electrical

In [76]:
print(station_equip_expenses)

[[2000, 0, 2000, 2000, 0, 0, 0, 2000, 2000], [351200, 83700, 351200, 351200, 1590, 14700, 1590, 351200, 351200], [1875, 1875, 1875, 1875, 525, 1500, 525, 1875, 1875], [6000, 6000, 6000, 6000, 0, 6000, 0, 6000, 6000], [332800, 83200, 332800, 332800, 0, 0, 0, 332800, 332800], [3000, 3000, 3000, 3000, 1000, 1000, 1000, 3000, 3000], [160, 240, 160, 160, 30, 60, 30, 160, 160], [37865.26, 0, 37865.26, 37865.26, 0, 0, 0, 37865.26, 37865.26], [0, 0, 0, 0, 0, 0, 0, 0, 8000], [8000, 3000, 8000, 8000, 3000, 3000, 3000, 8000, 8000]]


In [77]:
# Code to calculate total cost of station equipment expenses (components calculated above):
station_equip_expenses_total = []
total_contingencies = []

station_equip_len = len(station_equip_expenses[0])

for i in range(station_equip_len):
    station_equip_expenses_total.append(0)
    for j in station_equip_expenses:
        station_equip_expenses_total[i] += (j[i])

# Creating global variables:
for i in station_equip_expenses_total:
    total_contingencies.append(i * 0.05)
print(total_contingencies)
print(station_equip_expenses_total)

[37145.013, 9050.75, 37145.013, 37145.013, 307.25, 1313.0, 307.25, 37145.013, 37545.013]
[742900.26, 181015, 742900.26, 742900.26, 6145, 26260, 6145, 742900.26, 750900.26]


In [78]:
# NAME:         Equipment Installation
# DESCRIPTION:  Nonresidential structures

tier_einstall = []
type1_einstall = []
type2_einstall = []

for i in range(len(inputs)):
    tier_einstall.append(0)
    type1_einstall.append(0)
    type2_einstall.append(0)
    cost2008 = (station_equip_expenses_total[i] * 0.3) + (total_contingencies[i] * 0.125)
    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == "2332E0"].index[0]
    
    tier_einstall[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_einstall[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_einstall[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_einstall)
print(type1_einstall)
print(type2_einstall)

[1.8291600673733017, 0.40402858868287533, 1.6581661386069726, 1.4499919349950323, 0.010699527960187189, 0.05869674440459299, 0.013735395825065649, 1.4918218781680013, 1.5078867198000987]
[2.273828879751752, 0.5550140722736658, 2.277822824604398, 1.8826695283975499, 0.012566595589239853, 0.07267830892176705, 0.01700716711059629, 1.8755184652328114, 1.8957151841326845]
[3.1782481822410644, 1.0056261637491704, 4.127171441659868, 3.0139068609471638, 0.01737623150954418, 0.1036069939428091, 0.024244667851430386, 2.9837050303471506, 3.0158353734470134]


In [79]:
# NAME:         Site Prep & Construction
# DESCRIPTION:  Nonresidential structures

tier_siteprep = []
type1_siteprep = []
type2_siteprep = []

for i in range(len(inputs)):
    tier_siteprep.append(0)
    type1_siteprep.append(0)
    type2_siteprep.append(0)

    cost2008 = (station_equip_expenses_total[i] * 0.095) + (total_contingencies[i] * 0.75) + ((station_equip_expenses[3][0])/75)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == "2332E0"].index[0]
    
    tier_siteprep[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_siteprep[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_siteprep[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_siteprep)
print(type1_siteprep)
print(type2_siteprep)

[0.7920348864093743, 0.17538726326325096, 0.7179937134344929, 0.6278533071069501, 0.00508402139909939, 0.025979219033642042, 0.006526553933928174, 0.6459658686484281, 0.6529163715586417]
[0.9845785672954932, 0.24092948354472782, 0.9863079641449629, 0.815204733947664, 0.005971183133242234, 0.03216746900745558, 0.008081179080989865, 0.8121082900648039, 0.8208464623235244]
[1.3761963662463692, 0.4365384669952797, 1.7870845872341938, 1.3050358035022358, 0.008256544883002688, 0.04585652605371489, 0.01152017272438369, 1.2919582692310076, 1.305859560531356]


In [80]:
# NAME:         Electrical Infrastrastructure & Make Ready
# DESCRIPTION:  Power, distribution, and specialty transformer manufacturing

tier_einfra = []
type1_einfra = []
type2_einfra = []

for i in range(len(inputs)):
    tier_einfra.append(0)
    type1_einfra.append(0)
    type2_einfra.append(0)

    cost2008 = (station_equip_expenses_total[i] * 0.3) + (total_contingencies[i] * 0.125)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == 335311].index[0]
    
    tier_einfra[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_einfra[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_einfra[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_einfra)
print(type1_einfra)
print(type2_einfra)

[0.7555822490737789, 0.17505444337611964, 0.7184376515663041, 0.5603388519252084, 0.004583275157669017, 0.0, 0.0, 0.707975688527706, 0.7155995995870745]
[1.1676012494937105, 0.3195268754944312, 1.3113642454039751, 1.0078814929578723, 0.005690136108246084, 0.0, 0.0, 1.1169732437901618, 1.1290014882685275]
[1.6670725814946847, 0.5956969694030129, 2.444788738230038, 1.6888300715491025, 0.007993091665022817, 0.0, 0.0, 1.7737109090603487, 1.7928113025270072]


In [81]:
# NAME:         Engineering & Design
# DESCRIPTION:  Architectural, engineering, and related services

tier_eng = []
type1_eng = []
type2_eng = []

for i in range(len(inputs)):
    tier_eng.append(0)
    type1_eng.append(0)
    type2_eng.append(0)

    cost2008 = (station_equip_expenses_total[i] * 0.195)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == 541300].index[0]
    
    tier_eng[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_eng[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_eng[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_eng)
print(type1_eng)
print(type2_eng)

[0.7917895107416792, 0.16588078623656305, 0.6807882176844301, 0.6220764104635996, 0.003995320776261342, 0.02572620716412775, 0.006020089224050458, 0.6383160654955317, 0.6451898395388524]
[1.080713503211318, 0.2843030795308454, 1.1668029262893447, 1.0036580806419715, 0.005985789586994743, 0.03493361670816908, 0.008174679157338119, 1.0014540751559398, 1.0122383392525056]
[1.580029735079822, 0.543908784092327, 2.232245820061727, 1.6745597725741361, 0.008522966226534711, 0.052076597040347575, 0.01218624100582391, 1.6557530664213473, 1.6735832183873336]


In [82]:
# NAME:         Permitting
# DESCRIPTION:  Architectural, engineering, and related services

tier_permit = []
type1_permit = []
type2_permit = []

for i in range(len(inputs)):
    tier_permit.append(0)
    type1_permit.append(0)
    type2_permit.append(0)

    cost2008 = (station_equip_expenses_total[i] * 0.03)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == 541300].index[0]
    
    tier_permit[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_permit[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_permit[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_permit)
print(type1_permit)
print(type2_permit)

[0.12181377088333528, 0.02552012095947124, 0.1047366488745277, 0.09570406314824609, 0.0006146647348094372, 0.003957878025250423, 0.0009261675729308397, 0.09820247161469718, 0.0992599753136696]
[0.16626361587866434, 0.04373893531243775, 0.17950814250605304, 0.15440893548338025, 0.0009208907056914988, 0.00537440257048755, 0.0012576429472827874, 0.15406985771629841, 0.15572897526961624]
[0.24308149770458806, 0.0836782744757426, 0.34342243385565036, 0.25762458039602093, 0.001311225573313032, 0.008011784160053473, 0.0018748063085882937, 0.2547312409878996, 0.257474341290359]


In [83]:
# NAME:         Contingencies - Install
# DESCRIPTION:  Nonresidential structures

tier_cinstal = []
type1_cinstal = []
type2_cinstal = []

for i in range(len(inputs)):
    tier_cinstal.append(0)
    type1_cinstal.append(0)
    type2_cinstal.append(0)

    cost2008 = (total_contingencies[i] * 0.125)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == "2332E0"].index[0]
    
    tier_cinstal[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_cinstal[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_cinstal[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_cinstal)
print(type1_cinstal)
print(type2_cinstal)

[0.03732979729333269, 0.008245481401691334, 0.03384012527769332, 0.029591672142755765, 0.00021835771347320795, 0.0011978927429508773, 0.0002803142005115439, 0.03044534445240819, 0.030773198363267317]
[0.04640467101534187, 0.011326817801503384, 0.04648618009396731, 0.03842182711015408, 0.00025646113447428277, 0.0014832307943217766, 0.00034708504307339366, 0.03827588704556758, 0.03868806498229968]
[0.06486220780083805, 0.020522982933656536, 0.08422798860530344, 0.061508303284636, 0.00035461696958253436, 0.002114428447812431, 0.00049478913982511, 0.06089193939483981, 0.06154766068259211]


In [84]:
# NAME:         Contingencies - Site Prep & Construction
# DESCRIPTION:  Nonresidential structures

tier_csiteprep = []
type1_csiteprep = []
type2_csiteprep = []

for i in range(len(inputs)):
    tier_csiteprep.append(0)
    type1_csiteprep.append(0)
    type2_csiteprep.append(0)

    cost2008 = (total_contingencies[i] * 0.75)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == "2332E0"].index[0]
    
    tier_csiteprep[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_csiteprep[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_csiteprep[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_csiteprep)
print(type1_csiteprep)
print(type2_csiteprep)

[0.2239787837599961, 0.049472888410148, 0.2030407516661599, 0.17755003285653456, 0.0013101462808392476, 0.007187356457705265, 0.0016818852030692634, 0.1826720667144491, 0.1846391901796039]
[0.2784280260920512, 0.06796090680902031, 0.2789170805638038, 0.23053096266092446, 0.0015387668068456964, 0.008899384765930659, 0.0020825102584403616, 0.22965532227340543, 0.23212838989379805]
[0.38917324680502824, 0.1231378976019392, 0.5053679316318206, 0.3690498197078159, 0.0021277018174952057, 0.012686570686874584, 0.0029687348389506596, 0.36535163636903883, 0.36928596409555264]


In [85]:
# NAME:         Contingencies - Electrical
# DESCRIPTION:  Power, distribution, and specialty transformer manufacturing

tier_celec = []
type1_celec = []
type2_celec = []


for i in range(len(inputs)):
    tier_celec.append(0)
    type1_celec.append(0)
    type2_celec.append(0)

    cost2008 = (total_contingencies[i] * 0.125)

    cost2008 = cost2008 * empdef 
    row_num = tier_mult[tier_mult['Code'] == 335311].index[0]
    
    tier_celec[i] = ((cost2008 * (tier_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type1_celec[i] = ((cost2008 * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000)
    type2_celec[i] = ((cost2008 * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000)
        

print(tier_celec)
print(type1_celec)
print(type2_celec)

[0.015420045899464874, 0.003572539660737135, 0.014661992889108248, 0.011435486773983842, 9.353622770753097e-05, 0.0, 0.0, 0.014448483439340938, 0.0146040734609607]
[0.02382859692844307, 0.006520956642743493, 0.026762535620489285, 0.02056901006036474, 0.0001161252266988997, 0.0, 0.0, 0.0227953723222482, 0.023040846699357697]
[0.03402188941825887, 0.012157081008224754, 0.04989364771898037, 0.0344659198275327, 0.00016312431969434322, 0.0, 0.0, 0.036198181817558135, 0.03658798576585729]


## Station Development Tables
Finding the total values for each row using station development calc components.

#### Station Development Table

##### Performing calculations for Stations Development Employment Table

In [86]:
# Creating Row and Column headings that will be used in each dataframe:
#   ROW HEADINGS:    Contains RUN #
#   COLUMN HEADINGS: Contains total values that will be calculated - Tier, Type I, Type II, ...

row_headings_dev = []
for i in range(len(inputs)):
    row_headings_dev.append("RUN " + str(i + 1))

column_total_headings_dev = ["Direct Employment", "Indirect Employment", "Induced Employment", "Total Employment"]

print(row_headings_dev)
print(column_total_headings_dev)

['RUN 1', 'RUN 2', 'RUN 3', 'RUN 4', 'RUN 5', 'RUN 6', 'RUN 7', 'RUN 8', 'RUN 9']
['Direct Employment', 'Indirect Employment', 'Induced Employment', 'Total Employment']


In [87]:
# Calculating totals for all EVSE components:

tier_total = []
type1_total = []
type2_total = []
direct_emp_total = []
indirect_emp_total = []
induced_emp_total = []
station_dev_totals = []


for i in range(len(inputs)):
    # PART 1:
    # Equipment component totals:

    if inputs["Geography"][i] != "USA-National":
        ptier = ptier_trenching[i]
        ptype1 = ptype1_trenching[i]
        ptype2 = ptype2_trenching[i]
        wtier = wtier_trenching[i]
        wtype1 = wtype1_trenching[i]
        wtype2 = wtype2_trenching[i]
        stier = 0
        stype1 = 0
        stype2 = 0
    else:
        ptier = ptier_cable_cooling[i] + ptier_charger[i] + ptier_conduit_cables[i] + ptier_trenching[i] + ptier_electrical_storage[i] + ptier_safety_traffic[i] + ptier_load_center[i] + ptier_transformers[i] + ptier_meters[i] + ptier_misc[i]
        ptype1 = ptype1_cable_cooling[i] + ptype1_charger[i] + ptype1_conduit_cables[i] + ptype1_trenching[i] + ptype1_electrical_storage[i] + ptype1_safety_traffic[i] + ptype1_load_center[i] + ptype1_transformers[i] + ptype1_meters[i] + ptype1_misc[i]
        ptype2 = ptype2_cable_cooling[i] + ptype2_charger[i] + ptype2_conduit_cables[i] + ptype2_trenching[i] + ptype2_electrical_storage[i] + ptype2_safety_traffic[i] + ptype2_load_center[i] + ptype2_transformers[i] + ptype2_meters[i] + ptype2_misc[i]

        wtier = wtier_cable_cooling[i] + wtier_charger[i] + wtier_conduit_cables[i] + wtier_trenching[i] + wtier_electrical_storage[i] + wtier_safety_traffic[i] + wtier_load_center[i] + wtier_transformers[i] + wtier_meters[i] + wtier_misc[i]
        wtype1 = wtype1_cable_cooling[i] + wtype1_charger[i] + wtype1_conduit_cables[i] + wtype1_trenching[i] + wtype1_electrical_storage[i] + wtype1_safety_traffic[i] + wtype1_load_center[i] + wtype1_transformers[i] + wtype1_meters[i] + wtype1_misc[i]
        wtype2 = wtype2_cable_cooling[i] + wtype2_charger[i] + wtype2_conduit_cables[i] + wtype2_trenching[i] + wtype2_electrical_storage[i] + wtype2_safety_traffic[i] + wtype2_load_center[i] + wtype2_transformers[i] + wtype2_meters[i] + wtype2_misc[i]

        stier = stier_cable_cooling[i] + stier_charger[i] + stier_conduit_cables[i] + stier_electrical_storage[i] + stier_safety_traffic[i] + stier_load_center[i] + stier_transformers[i] + stier_meters[i] + stier_misc[i]
        stype1 = stype1_cable_cooling[i] + stype1_charger[i] + stype1_conduit_cables[i] + stype1_electrical_storage[i] + stype1_safety_traffic[i] + stype1_load_center[i] + stype1_transformers[i] + stype1_meters[i] + stype1_misc[i]
        stype2 = stype2_cable_cooling[i] + stype2_charger[i] + stype2_conduit_cables[i] + stype2_electrical_storage[i] + stype2_safety_traffic[i] + stype2_load_center[i] + stype2_transformers[i] + stype2_meters[i] + stype2_misc[i]
 
    # Non-Equipment component totals:
    netier = tier_einstall[i] + tier_siteprep[i] + tier_einfra[i] + tier_eng[i] + tier_permit[i] + tier_cinstal[i] + tier_csiteprep[i] + tier_celec[i]
    netype1 = type1_einstall[i] + type1_siteprep[i] + type1_einfra[i] + type1_eng[i] + type1_permit[i] + type1_cinstal[i] + type1_csiteprep[i] + type1_celec[i]
    netype2 = type2_einstall[i] + type2_siteprep[i] + type2_einfra[i] + type2_eng[i] + type2_permit[i] + type2_cinstal[i] + type2_csiteprep[i] + type2_celec[i]

    # PART 2:
    # Calculating total Tier, Type I, Type II Employment totals:
    tier_total.append((ptier + wtier + stier + netier) * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))
    type1_total.append((ptype1 + wtype1 + stype1 + netype1) * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))
    type2_total.append((ptype2 + wtype2 + stype2 + netype2) * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))
    
   
    
    # Calculating Direct, Indirect, and Induced Employment totals:
    direct_emp_total.append(tier_total[i])
    if str(inputs["yes/no to include indirect effects"][i]).strip().lower() == "yes":
        indirect_emp_total.append(type1_total[i] - tier_total[i])
    else:
        indirect_emp_total.append(0)
    induced_emp_total.append(type2_total[i] - type1_total[i])
    
    
# Putting together in total table:
for i in range(len(type1_total)):
    curr_row = []
    curr_row.append(type1_total[i])
    curr_row.append(induced_emp_total[i])
    curr_row.append(type2_total[i])
    station_dev_totals.append(curr_row)

    
print(tier_total)
print(type1_total)
print(type2_total)
print(direct_emp_total)
print(indirect_emp_total)
print(induced_emp_total)
print(station_dev_totals)


[46.34275468585627, 15.235747241154005, 61.16611858245925, 108.83316717740743, 1.0639540100018543, 1.8496662404288928, 0.29170405959555934, 38.64680224843727, 19.528506627926667]
[61.12617999516306, 26.039889607739433, 104.92346298595828, 156.9791405911181, 1.3218379316573317, 2.3950645530481967, 0.3695026359772082, 53.30218193697557, 26.93377715459317]
[86.6146057025128, 48.11526047926114, 193.99320804672607, 256.0001457426804, 1.8442201185675802, 3.4481783126041607, 0.5328941186900206, 85.49858339870951, 43.202717364844155]
[46.34275468585627, 15.235747241154005, 61.16611858245925, 108.83316717740743, 1.0639540100018543, 1.8496662404288928, 0.29170405959555934, 38.64680224843727, 19.528506627926667]
[14.783425309306793, 0, 43.75734440349903, 48.145973413710664, 0.25788392165547736, 0, 0, 14.655379688538304, 7.405270526666502]
[25.48842570734974, 22.075370871521706, 89.06974506076779, 99.02100515156229, 0.5223821869102485, 1.053113759555964, 0.16339148271281234, 32.19640146173394, 16.

Creating Dataframe with total output values

In [88]:
# Creating Row and Column headings that will be used in each dataframe:
#   ROW HEADINGS:    Contains RUN #
#   COLUMN HEADINGS: Contains total values that will be calculated - Tier, Type I, Type II, ...

row_headings_statdev = []
for i in range(len(inputs)):
    row_headings_statdev.append("RUN " + str(i + 1))

column_total_headings_statdev = ["Supply Chain Employment", "Induced Employment", "Total Employment"]

print(row_headings_statdev)
print(column_total_headings_statdev)

['RUN 1', 'RUN 2', 'RUN 3', 'RUN 4', 'RUN 5', 'RUN 6', 'RUN 7', 'RUN 8', 'RUN 9']
['Supply Chain Employment', 'Induced Employment', 'Total Employment']


##### Final Table:

In [89]:
statdev_df = pd.DataFrame(station_dev_totals, index = row_headings_statdev, columns = column_total_headings_statdev)
statdev_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,61.126180,25.488426,86.614606
RUN 2,26.039890,22.075371,48.115260
RUN 3,104.923463,89.069745,193.993208
RUN 4,156.979141,99.021005,256.000146
RUN 5,1.321838,0.522382,1.844220
RUN 6,2.395065,1.053114,3.448178
RUN 7,0.369503,0.163391,0.532894
RUN 8,53.302182,32.196401,85.498583
RUN 9,26.933777,16.268940,43.202717


#### Civil Construction Employment Table:
Includes Equipment Installation and Site Prep & Construction and takes total values for direct, indirect, and induced employment

##### Performing calculations for Civil Construction Employment

In [90]:
civil_const_emp = []
civil_const_direct = []
civil_const_indirect = []
civil_const_induced = []
civil_const_totals = []

for i in range(len(inputs)):
    direct = 0
    indirect = 0
    induced = 0
    # Calculating direct employment:
    direct = tier_einstall[i] + tier_siteprep[i] 
    civil_const_direct.append(direct * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

    # Calculating indirect employment:
    if str(inputs["yes/no to include indirect effects"][i]).strip().lower() == "yes":
        indirect = (type1_einstall[i] + type1_siteprep[i]) - (direct)
    else:
        indirect = 0
    civil_const_indirect.append(indirect * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

    # Calculating induced employment:
    induced = (type2_einstall[i] + type2_siteprep[i]) - (type1_einstall[i] + type1_siteprep[i]) 
    civil_const_induced.append(induced * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

    # Calculating total and appending to civil_const_emp list per row:
    civil_const_emp.append((direct + indirect + induced) * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

# Appending total values to row for dataframe
for i in range(len(civil_const_direct)):
    curr_row = []
    curr_row.append(civil_const_direct[i])
    curr_row.append(civil_const_indirect[i])
    curr_row.append(civil_const_induced[i])
    curr_row.append(civil_const_emp[i])
    civil_const_totals.append(curr_row)

# Verify output
print(civil_const_direct)
print(civil_const_indirect)
print(civil_const_induced)
print(civil_const_emp)
print(civil_const_totals)



[26.21194953782676, 5.794158519461263, 23.76159852041465, 62.33535726305948, 0.6313419743714632, 0.8467596343823504, 0.20261949758993825, 21.377877468164293, 10.804015456793703]
[6.372124932645691, 0.0, 8.87970936707896, 18.600870607296937, 0.11016917452782021, 0.0, 0.0, 5.498390084811859, 2.778792775487342]
[12.960371014401883, 6.462210749260564, 26.50125240144701, 48.63205206312557, 0.2837999068025912, 0.4461774206730136, 0.10676494384227922, 15.88036544280543, 8.025666437610802]
[45.54444548487433, 12.256369268721826, 59.14256028894062, 129.56827993348196, 1.0253110557018745, 1.292937055055364, 0.30938444143221744, 42.75663299578159, 21.608474669891848]
[[26.21194953782676, 6.372124932645691, 12.960371014401883, 45.54444548487433], [5.794158519461263, 0.0, 6.462210749260564, 12.256369268721826], [23.76159852041465, 8.87970936707896, 26.50125240144701, 59.14256028894062], [62.33535726305948, 18.600870607296937, 48.63205206312557, 129.56827993348196], [0.6313419743714632, 0.1101691745

##### Creating Dataframe with total output values

In [91]:
civil_const_df = pd.DataFrame(civil_const_totals, index = row_headings_dev, columns = column_total_headings_dev)
civil_const_df

,Direct Employment,Indirect Employment,Induced Employment,Total Employment
RUN 1,26.211950,6.372125,12.960371,45.544445
RUN 2,5.794159,0.000000,6.462211,12.256369
RUN 3,23.761599,8.879709,26.501252,59.142560
RUN 4,62.335357,18.600871,48.632052,129.568280
RUN 5,0.631342,0.110169,0.283800,1.025311
RUN 6,0.846760,0.000000,0.446177,1.292937
RUN 7,0.202619,0.000000,0.106765,0.309384
RUN 8,21.377877,5.498390,15.880365,42.756633
RUN 9,10.804015,2.778793,8.025666,21.608475


#### Electrical Construction Employment
Includes Electrical Infrastrastructure & Make Ready and takes total values for direct, indirect, and induced employment

##### Performing calculations for Electrical Construction Employment

In [92]:
# NOTE: Does not include induced costs in actual tool (typo - Station Operation Calc not Station Development Calc)


elec_const_emp = []
elec_const_direct = []
elec_const_indirect = []
elec_const_induced = []
elec_const_totals = []

for i in range(len(inputs)):
    # Calculating direct employment:
    direct =  tier_einfra[i] 
    elec_const_direct.append(direct * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

    # Calculating indirect employment:
    if str(inputs["yes/no to include indirect effects"][i]).strip().lower() == "yes":
        indirect = (type1_einfra[i]) - (direct)
    else:
        indirect = 0
    elec_const_indirect.append(indirect * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

    # Calculating induced employment:
    induced = (type2_einfra[i]) - (type1_einfra[i]) 
    elec_const_induced.append(induced * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i]))

    # Calculating total and appending to civil_const_emp list per row:
    elec_const_emp.append((direct + indirect + induced)  * inputs["Number of years for analysis"][i] * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])) #### TOOK OUT INDUCED TO CHECK RESULTS, PUT BACK IN AFTER

# Appending total values to row for dataframe
for i in range(len(elec_const_direct)):
    curr_row = []
    curr_row.append(elec_const_direct[i])
    curr_row.append(elec_const_indirect[i])
    curr_row.append(elec_const_induced[i])
    curr_row.append(elec_const_emp[i])
    elec_const_totals.append(curr_row)

    
print(elec_const_direct)
print(elec_const_indirect)
print(elec_const_induced)
print(elec_const_emp)
print(elec_const_totals)

[7.555822490737789, 1.7505444337611964, 7.1843765156630415, 16.81016555775625, 0.1833310063067607, 0.0, 0.0, 7.07975688527706, 3.5779979979353724]
[4.120190004199316, 0.0, 5.92926593837671, 13.426279230979917, 0.04427443802308268, 0.0, 0.0, 4.0899755526245585, 2.0670094434072652]
[4.994713320009742, 2.7617009390858165, 11.334244928260627, 20.428457357736903, 0.09211822227106932, 0.0, 0.0, 6.567376652701869, 3.3190490712923983]
[16.67072581494685, 4.512245372847013, 24.44788738230038, 50.66490214647307, 0.31972366660091267, 0.0, 0.0, 17.737109090603486, 8.964056512635036]
[[7.555822490737789, 4.120190004199316, 4.994713320009742, 16.67072581494685], [1.7505444337611964, 0.0, 2.7617009390858165, 4.512245372847013], [7.1843765156630415, 5.92926593837671, 11.334244928260627, 24.44788738230038], [16.81016555775625, 13.426279230979917, 20.428457357736903, 50.66490214647307], [0.1833310063067607, 0.04427443802308268, 0.09211822227106932, 0.31972366660091267], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 

##### Creating Dataframe with total output values

In [93]:
elec_const_df = pd.DataFrame(elec_const_totals, index = row_headings_dev, columns = column_total_headings_dev)
elec_const_df

,Direct Employment,Indirect Employment,Induced Employment,Total Employment
RUN 1,7.555822,4.120190,4.994713,16.670726
RUN 2,1.750544,0.000000,2.761701,4.512245
RUN 3,7.184377,5.929266,11.334245,24.447887
RUN 4,16.810166,13.426279,20.428457,50.664902
RUN 5,0.183331,0.044274,0.092118,0.319724
RUN 6,0.000000,0.000000,0.000000,0.000000
RUN 7,0.000000,0.000000,0.000000,0.000000
RUN 8,7.079757,4.089976,6.567377,17.737109
RUN 9,3.577998,2.067009,3.319049,8.964057


## Station Operation Formulas and Tables
Formulas for all EVSE Components that have to do with Station Operation
Calculating total values based on formulas and multipliers for each JOBS EVSE Component.

In [94]:
# Creating Row and Column headings that will be used in each dataframe:
#   ROW HEADINGS:    Contains RUN #
#   COLUMN HEADINGS: Contains total values that will be calculated - Tier, Type I, Type II, ...

row_headings_ops = []
for i in range(len(inputs)):
    row_headings_ops.append("RUN " + str(i + 1))

column_total_headings_ops = ["Supply Chain Employment", "Induced Employment", "Total Employment"]

print(row_headings_ops)
print(column_total_headings_ops)

['RUN 1', 'RUN 2', 'RUN 3', 'RUN 4', 'RUN 5', 'RUN 6', 'RUN 7', 'RUN 8', 'RUN 9']
['Supply Chain Employment', 'Induced Employment', 'Total Employment']


In [95]:
# Creating a dictionary that maps inputs["[Average] amount of kWh dispensed per charge session"][i] to more precise values:
kWh_dict = {6.6: 6.60000,
            14.3: 14.29804,
            25.2: 25.16471,
            52.9: 52.9, 
            31.6: 31.6, 
            17.5: 17.5}

kWh_dict[14.3]


14.29804

#### Electriticy Sector Employment
Includes Electrical Infrastrastructure & Make Ready and takes total values for supply chain and induced employment

##### Test

In [96]:
ans = 172126.59 * 3.57045E-07 
sum = 0
print(ans * 2)
total = 0
for i in range(5):
    sum = ans * (i + 1)
    print(sum)
    total += sum

print(total)
print("this")
print(ans * 5)
for i in range(5):
    total = total + (ans * 5)
print(total)

0.1229138766531
0.06145693832655
0.1229138766531
0.18437081497965
0.2458277533062
0.30728469163274996
0.9218540748982499
this
0.30728469163274996
2.4582775330619993


In [97]:
ans = 152*52.9 * 12
ans * 2.9675E-07 

0.028633288800000004

##### Performing calculations for Electricity Sector Employment

In [98]:
# NOTE: CHANGE Average amount of kWh dispensed per charge session to more accurate numbers

elec_sec_emp = []
elec_type1_sum = 0
elec_type1_sum_list = []
elec_induced_sum = 0
elec_induced_sum_list = []
elec_sec_totals = []

for i in range(len(inputs)):
    # Calculating supply chain employment:
    if inputs["Charger Power"][i] < 10.9:
        elec_cost = elec_rate[inputs["Geography"][i]][0]
    else:
        elec_cost = elec_rate[inputs["Geography"][i]][1]

        
    elec_in_region = elec_cost * empdef
    row_num = tier_mult[tier_mult['Code'] == "2211A0"].index[0]

    type1_elec_in_region = ((elec_in_region * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type2_elec_in_region = ((elec_in_region * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    
    avg_kwh = 0
    if inputs["[Average] amount of kWh dispensed per charge session"][i] in kWh_dict.keys():
        avg_kwh = kWh_dict[inputs["[Average] amount of kWh dispensed per charge session"][i]]
    else:
        avg_kwh = inputs["[Average] amount of kWh dispensed per charge session"][i]

    type1_total_elec_in_region = type1_elec_in_region * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    
    if str(inputs["yes/no to include induced effects"][i]).strip().lower() == "yes":
        induced_total_elec_region = ((type2_elec_in_region - type1_elec_in_region) * inputs["[Average] sessions per month"][i] * avg_kwh * 12)
    else:
        induced_total_elec_region = 0

    elec_type1_sum = 0
    elec_induced_sum = 0
    
    for j in range((inputs["Number of years for analysis"][i])):
        elec_type1_sum += type1_total_elec_in_region * (j+1)
        elec_induced_sum += induced_total_elec_region * (j+1)

    # if ((inputs["Number of years for analysis"][i]) < 10):
    #     elec_type1_sum += (type1_total_elec_in_region * (11 - (inputs["Number of years for analysis"][i])))
    #     elec_induced_sum += (induced_total_elec_region * (11 - (inputs["Number of years for analysis"][i])))
    
    elec_sector_total = elec_type1_sum + elec_induced_sum
    elec_type1_sum_list.append(elec_type1_sum)
    elec_induced_sum_list.append(elec_induced_sum)
    elec_sec_emp.append(elec_sector_total)

for i in range(len(elec_type1_sum_list)):
    curr_row = []
    curr_row.append(elec_type1_sum_list[i])
    curr_row.append(elec_induced_sum_list[i])
    curr_row.append(elec_sec_emp[i])
    elec_sec_totals.append(curr_row)

print(elec_type1_sum_list)
print(elec_induced_sum_list)
print(elec_sec_emp)
print(elec_sec_totals)



[0.5947394042862837, 0.4506837328335074, 0.9013674656670148, 1.7184984296078327, 0.23531542418348433, 0.12985675853410428, 0.07144768603767979, 1.5748320130855442, 0.42949963993242124]
[0.3333470468482329, 0, 0.9754178640044762, 1.3787602323821813, 0, 0.0765344847863046, 0.04210956673950363, 1.2518534879413614, 0.34141458762037125]
[0.9280864511345166, 0.4506837328335074, 1.876785329671491, 3.097258661990014, 0.23531542418348433, 0.20639124332040887, 0.11355725277718343, 2.8266855010269056, 0.7709142275527925]
[[0.5947394042862837, 0.3333470468482329, 0.9280864511345166], [0.4506837328335074, 0, 0.4506837328335074], [0.9013674656670148, 0.9754178640044762, 1.876785329671491], [1.7184984296078327, 1.3787602323821813, 3.097258661990014], [0.23531542418348433, 0, 0.23531542418348433], [0.12985675853410428, 0.0765344847863046, 0.20639124332040887], [0.07144768603767979, 0.04210956673950363, 0.11355725277718343], [1.5748320130855442, 1.2518534879413614, 2.8266855010269056], [0.4294996399324

##### Creating Dataframe with total output values

In [99]:
elec_sect_df = pd.DataFrame(elec_sec_totals, index = row_headings_ops, columns = column_total_headings_ops)
elec_sect_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,0.594739,0.333347,0.928086
RUN 2,0.450684,0.000000,0.450684
RUN 3,0.901367,0.975418,1.876785
RUN 4,1.718498,1.378760,3.097259
RUN 5,0.235315,0.000000,0.235315
RUN 6,0.129857,0.076534,0.206391
RUN 7,0.071448,0.042110,0.113557
RUN 8,1.574832,1.251853,2.826686
RUN 9,0.429500,0.341415,0.770914


#### Retail Sector Employment
Includes Induced Purchases and takes total values for supply chain and induced employment

##### Performing calculations for Retail Sector Employment

In [100]:
# Creating a dictionary that maps inputs["Retail dollars per session"][i] to more precise values:
retail_dict = {0:0,
               0.40: 0.397381954,
               0.70: 0.699396599,
               0.19: 0.189036}

retail_dict[0.4]

0.397381954

In [101]:
retail_sect_emp = []
retail_type1_sum = 0
retail_type1_sum_list = []
retail_induced_sum = 0
retail_induced_sum_list = []
retail_sect_totals = []


for i in range(len(inputs)):
    avg_kwh = 0
    if inputs["[Average] amount of kWh dispensed per charge session"][i] in kWh_dict.keys():
        avg_kwh = kWh_dict[inputs["[Average] amount of kWh dispensed per charge session"][i]]
    else:
        avg_kwh = inputs["[Average] amount of kWh dispensed per charge session"][i]

    retail_rev = (inputs["Retail dollars per session"][i]) / (inputs["[Average] sessions per month"][i] * avg_kwh)

    retail_in_region = retail_rev * empdef
    row_num = tier_mult[tier_mult['Code'] == 445000].index[0]
    type1_retail_in_region = ((retail_in_region * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type2_retail_in_region = ((retail_in_region * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])

    type1_total_retail_in_region = type1_retail_in_region * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    
    if str(inputs["yes/no to include induced effects"][i]).strip().lower() == "yes":
        induced_total_retail_region = ((type2_retail_in_region - type1_retail_in_region) * inputs["[Average] sessions per month"][i] * avg_kwh * 12)
    else:
        induced_total_retail_region = 0

    retail_type1_sum = 0
    retail_induced_sum = 0
    for j in range((inputs["Number of years for analysis"][i])):
        retail_type1_sum += type1_total_retail_in_region * (j+1)
        retail_induced_sum += induced_total_retail_region * (j+1)

    retail_sector_total = retail_type1_sum + retail_induced_sum
    retail_type1_sum_list.append(retail_type1_sum)
    retail_induced_sum_list.append(retail_induced_sum)
    retail_sect_emp.append(retail_sector_total)

for i in range(len(retail_type1_sum_list)):
    curr_row = []
    curr_row.append(retail_type1_sum_list[i])
    curr_row.append(retail_induced_sum_list[i])
    curr_row.append(retail_sect_emp[i])
    retail_sect_totals.append(curr_row)

print(retail_type1_sum_list)
print(retail_induced_sum_list)
print(retail_sect_emp)
print(retail_sect_totals)


[0.003829346113864657, 0.0, 0.0038295005959517996, 0.01084179563333143, 0.0, 0.0067374658871330595, 0.0, 13.39304964741892, 3.652649903841523]
[0.000778126272944328, 0, 0.0016053999184700398, 0.0030705963084221057, 0, 0.001344225881283515, 0.0, 3.793484200777656, 1.0345866002120883]
[0.004607472386808985, 0.0, 0.00543490051442184, 0.013912391941753536, 0.0, 0.008081691768416575, 0.0, 17.186533848196575, 4.687236504053612]
[[0.003829346113864657, 0.000778126272944328, 0.004607472386808985], [0.0, 0, 0.0], [0.0038295005959517996, 0.0016053999184700398, 0.00543490051442184], [0.01084179563333143, 0.0030705963084221057, 0.013912391941753536], [0.0, 0, 0.0], [0.0067374658871330595, 0.001344225881283515, 0.008081691768416575], [0.0, 0.0, 0.0], [13.39304964741892, 3.793484200777656, 17.186533848196575], [3.652649903841523, 1.0345866002120883, 4.687236504053612]]


##### Creating Dataframe with total output values

In [102]:
retail_sect_df = pd.DataFrame(retail_sect_totals, index = row_headings_ops, columns = column_total_headings_ops)
retail_sect_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,0.003829,0.000778,0.004607
RUN 2,0.000000,0.000000,0.000000
RUN 3,0.003830,0.001605,0.005435
RUN 4,0.010842,0.003071,0.013912
RUN 5,0.000000,0.000000,0.000000
RUN 6,0.006737,0.001344,0.008082
RUN 7,0.000000,0.000000,0.000000
RUN 8,13.393050,3.793484,17.186534
RUN 9,3.652650,1.034587,4.687237


#### Advertising Sector Employment
Includes Advertisements and takes total values for supply chain and induced employment

##### Performing calculations for Advertising Sector Employment

In [103]:
# Creating a dictionary that maps inputs["Advertising"][i] to more precise values:
ad_dict = {0:0, 
           1.55: 1.554214664,
            0.49: 0.488012926,
            0.37: 0.366009695,
            0.18: 0.183004847}

ad_dict[0.37]


0.366009695

In [104]:
ad_emp = []
ad_type1_sum = 0
ad_type1_sum_list = []
ad_induced_sum = 0
ad_induced_sum_list = []
ad_totals = []

for i in range(len(inputs)):
    avg_kwh = 0
    if inputs["[Average] amount of kWh dispensed per charge session"][i] in kWh_dict.keys():
        avg_kwh = kWh_dict[inputs["[Average] amount of kWh dispensed per charge session"][i]]
    else:
        avg_kwh = inputs["[Average] amount of kWh dispensed per charge session"][i]

    ad_rev = (inputs["Advertising"][i]) / (inputs["[Average] sessions per month"][i] * avg_kwh)

    ad_in_region = ad_rev * empdef
    row_num = tier_mult[tier_mult['Code'] == 541800].index[0]
    type1_ad_in_region = ((ad_in_region * (type1_mult[inputs["Geography"][i]][row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type2_ad_in_region = ((ad_in_region * (type2_mult[inputs["Geography"][i]][row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])

    type1_total_ad_in_region = type1_ad_in_region * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    
    if str(inputs["yes/no to include induced effects"][i]).strip().lower() == "yes":
        induced_total_ad_region = ((type2_ad_in_region - type1_ad_in_region) * inputs["[Average] sessions per month"][i] * avg_kwh * 12)
    else:
        induced_total_ad_region = 0
    
    ad_type1_sum = 0
    ad_induced_sum = 0
    for j in range((inputs["Number of years for analysis"][i])):
        ad_type1_sum += type1_total_ad_in_region * (j+1)
        ad_induced_sum += induced_total_ad_region * (j+1)

    ad_sector_total = ad_type1_sum + ad_induced_sum
    ad_type1_sum_list.append(ad_type1_sum)
    ad_induced_sum_list.append(ad_induced_sum)
    ad_emp.append(ad_sector_total)

for i in range(len(ad_type1_sum_list)):
    curr_row = []
    curr_row.append(ad_type1_sum_list[i])
    curr_row.append(ad_induced_sum_list[i])
    curr_row.append(ad_emp[i])
    ad_totals.append(curr_row)

print(ad_type1_sum_list)
print(ad_induced_sum_list)
print(ad_emp)
print(ad_totals)


[0.001650611308159691, 0.0, 0.0016393225296416434, 0.003923166947439387, 0.0, 0.006499538645453006, 0.0, 5.131277006575104, 1.399439183611392]
[0.0005511291906514548, 0, 0.0012461137560166224, 0.002314117941382308, 0, 0.002268334967970225, 0.0, 2.912528029969403, 0.7943258263552918]
[0.002201740498811146, 0.0, 0.002885436285658266, 0.006237284888821695, 0.0, 0.008767873613423231, 0.0, 8.043805036544507, 2.193765009966684]
[[0.001650611308159691, 0.0005511291906514548, 0.002201740498811146], [0.0, 0, 0.0], [0.0016393225296416434, 0.0012461137560166224, 0.002885436285658266], [0.003923166947439387, 0.002314117941382308, 0.006237284888821695], [0.0, 0, 0.0], [0.006499538645453006, 0.002268334967970225, 0.008767873613423231], [0.0, 0.0, 0.0], [5.131277006575104, 2.912528029969403, 8.043805036544507], [1.399439183611392, 0.7943258263552918, 2.193765009966684]]


##### Creating Dataframe with total output values

In [105]:
ad_sect_df = pd.DataFrame(ad_totals, index = row_headings_ops, columns = column_total_headings_ops)
ad_sect_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,0.001651,0.000551,0.002202
RUN 2,0.000000,0.000000,0.000000
RUN 3,0.001639,0.001246,0.002885
RUN 4,0.003923,0.002314,0.006237
RUN 5,0.000000,0.000000,0.000000
RUN 6,0.006500,0.002268,0.008768
RUN 7,0.000000,0.000000,0.000000
RUN 8,5.131277,2.912528,8.043805
RUN 9,1.399439,0.794326,2.193765


#### Data and Networking Sector Employment
Includes Data Fees & Networking Fees and takes total values for supply chain and induced employment

##### Performing calculations for Data and Networking Sector Employment

In [106]:
data_sec_emp = []
data_type1_sum = 0
data_type1_sum_list = []
data_induced_sum = 0
data_induced_sum_list = []
data_totals = []

for i in range(len(inputs)):
    # Calculating supply chain employment:
    if inputs["Charger Power"][i] <= 10.9 and inputs["Number of chargers per station"][i] == 1:
        data_cost = 0
    elif inputs["Charger Power"][i] < 10.9 and inputs["Number of chargers per station"][i] == 3:
        data_cost = 0.07575758 
    elif inputs["Charger Power"][i] <= 10.9 and inputs["[Average] sessions per month"][i] == 63:
        data_cost = 0.03330460 
    elif inputs["Charger Power"][i] <= 10.9 and inputs["[Average] sessions per month"][i] == 46:
        data_cost = 0.04561282 
    elif inputs["Charger Power"][i] == 50 and inputs["[Average] sessions per month"][i] == 76:
        data_cost = 0.03137226  
    elif inputs["Charger Power"][i] == 50 and inputs["[Average] sessions per month"][i] == 114:
        data_cost = 0.02091484   
    elif inputs["Charger Power"][i] == 150 :
        data_cost = 0.01568613 
    elif inputs["Charger Power"][i] == 350 :
        data_cost = 0.00784306  
    
    datanetwork_in_region = data_cost * empdef * 0.5
    
    data_row_num = tier_mult[tier_mult['Code'] == 518200].index[0]
    network_row_num = tier_mult[tier_mult['Code'] == 517210].index[0]

    type1_data_in_region = ((datanetwork_in_region * (type1_mult[inputs["Geography"][i]][data_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type1_network_in_region = ((datanetwork_in_region * (type1_mult[inputs["Geography"][i]][network_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])

    type2_data_in_region = ((datanetwork_in_region * (type2_mult[inputs["Geography"][i]][data_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type2_network_in_region = ((datanetwork_in_region * (type2_mult[inputs["Geography"][i]][network_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])


    avg_kwh = 0
    if inputs["[Average] amount of kWh dispensed per charge session"][i] in kWh_dict.keys():
        avg_kwh = kWh_dict[inputs["[Average] amount of kWh dispensed per charge session"][i]]
    else:
        avg_kwh = inputs["[Average] amount of kWh dispensed per charge session"][i]

    type1_total_datanetwork_in_region = (type1_data_in_region + type1_network_in_region) * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    
    if str(inputs["yes/no to include induced effects"][i]).strip().lower() == "yes":
        induced_total_datanetwork_region = ((type2_data_in_region + type2_network_in_region) - (type1_data_in_region + type1_network_in_region)) * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    else:
        induced_total_datanetwork_region = 0
        
    
    data_type1_sum = 0
    data_induced_sum = 0
    for j in range((inputs["Number of years for analysis"][i])):
        data_type1_sum += type1_total_datanetwork_in_region * (j+1)
        data_induced_sum += induced_total_datanetwork_region * (j+1)
    
    data_sector_total = data_type1_sum + data_induced_sum
    data_sec_emp.append(data_sector_total)
    data_type1_sum_list.append(data_type1_sum)
    data_induced_sum_list.append(data_induced_sum)

for i in range(len(data_type1_sum_list)):
    curr_row = []
    curr_row.append(data_type1_sum_list[i])
    curr_row.append(data_induced_sum_list[i])
    curr_row.append(data_sec_emp[i])
    data_totals.append(curr_row)

print(data_type1_sum_list)
print(data_induced_sum_list)
print(data_sec_emp)
print(data_totals)


[0.1852179865640559, 0.24457056674689703, 0.24457056674689703, 0.5837586701276132, 0.3184538060459722, 0.08173868379399081, 0.08173868377221565, 0.40099116168429544, 0.10936122591389877]
[0.07211225113377465, 0, 0.19056913667228884, 0.3401166512343255, 0, 0.03439185244640851, 0.03439185243724653, 0.23131442397575336, 0.06308575199338728]
[0.25733023769783053, 0.24457056674689703, 0.43513970341918584, 0.9238753213619386, 0.3184538060459722, 0.11613053624039932, 0.11613053620946218, 0.6323055856600488, 0.17244697790728605]
[[0.1852179865640559, 0.07211225113377465, 0.25733023769783053], [0.24457056674689703, 0, 0.24457056674689703], [0.24457056674689703, 0.19056913667228884, 0.43513970341918584], [0.5837586701276132, 0.3401166512343255, 0.9238753213619386], [0.3184538060459722, 0, 0.3184538060459722], [0.08173868379399081, 0.03439185244640851, 0.11613053624039932], [0.08173868377221565, 0.03439185243724653, 0.11613053620946218], [0.40099116168429544, 0.23131442397575336, 0.63230558566004

##### Creating Dataframe with total output values

In [107]:
data_sect_df = pd.DataFrame(data_totals, index = row_headings_ops, columns = column_total_headings_ops)
data_sect_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,0.185218,0.072112,0.257330
RUN 2,0.244571,0.000000,0.244571
RUN 3,0.244571,0.190569,0.435140
RUN 4,0.583759,0.340117,0.923875
RUN 5,0.318454,0.000000,0.318454
RUN 6,0.081739,0.034392,0.116131
RUN 7,0.081739,0.034392,0.116131
RUN 8,0.400991,0.231314,0.632306
RUN 9,0.109361,0.063086,0.172447


#### Warranty, Maintenance, Administrative Costs, & Access Fees Sector Employment														 
Includes Data Fees & Networking Fees and takes total values for supply chain and induced employment

##### Performing calculations for Warranty, Maintenance, Administrative Costs, & Access Fees Sector Employment

In [108]:
warr_sec_emp = []
warr_type1_sum = 0
warr_type1_sum_list = []
warr_induced_sum = 0
warr_induced_sum_list = []
warr_totals = []

for i in range(len(inputs)):
    # Calculating supply chain employment:
    if inputs["Charger Power"][i] <= 10.9 and inputs["Number of chargers per station"][i] == 1:
        admin = 0
    elif inputs["Charger Power"][i] < 10.9 and inputs["Number of chargers per station"][i] == 3:
        admin = 0.025252525  
    elif inputs["Charger Power"][i] <= 10.9 and inputs["[Average] sessions per month"][i] == 63:
        admin = 0.011101533  
    elif inputs["Charger Power"][i] <= 10.9 and inputs["[Average] sessions per month"][i] == 46:
        admin = 0.015204274  
    elif inputs["Charger Power"][i] == 50 and inputs["[Average] sessions per month"][i] == 76:
        admin = 0.005228710   
    elif inputs["Charger Power"][i] == 50 and inputs["[Average] sessions per month"][i] == 114:
        admin = 0.003485807    
    elif inputs["Charger Power"][i] == 150 :
        admin = 0.002614355  
    elif inputs["Charger Power"][i] == 350 :
        admin = 0.001307177 
    
    if inputs["Charger Power"][i] == 6.6 and inputs["Number of chargers per station"][i] == 1:
        maint =  0.025252525 
        warr =  0.050505051 
    elif inputs["Charger Power"][i] == 6.6 and inputs["Number of chargers per station"][i] == 3: 
        maint =  0.012626263 
        warr =  0.025252525 
    elif inputs["Charger Power"][i] == 10.9 and inputs["[Average] sessions per month"][i] == 30: 
        maint = 0.011656610  
        warr =  0.023313220
    elif inputs["Charger Power"][i] == 10.9 and inputs["[Average] sessions per month"][i] == 63: 
        maint = 0.005550767  
        warr = 0.011101533 
    elif inputs["Charger Power"][i] == 10.9 and inputs["[Average] sessions per month"][i] == 46: 
        maint = 0.007602137   
        warr = 0.015204274 
    elif inputs["Charger Power"][i] == 50 and inputs["[Average] sessions per month"][i] == 76: 
        maint = 0.002614355  
        warr = 0.005228710  
    elif inputs["Charger Power"][i] == 50 and inputs["[Average] sessions per month"][i] == 114: 
        maint = 0.001742903 
        warr = 0.003485807  
    elif inputs["Charger Power"][i] == 150 :
        maint = 0.001307177  
        warr = 0.002614355  
    elif inputs["Charger Power"][i] == 350 :
        maint = 0.000653589 
        warr = 0.001307177  
     
    
    admin_in_region = admin * empdef
    maint_in_region = maint * empdef
    warr_in_region = warr * empdef

    
    admin_row_num = tier_mult[tier_mult['Code'] == 561100].index[0]
    maint_row_num = tier_mult[tier_mult['Code'] == 811300].index[0]
    warr_row_num = tier_mult[tier_mult['Code'] == "5241XX"].index[0]

    type1_admin_in_region = ((admin_in_region * (type1_mult[inputs["Geography"][i]][admin_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type1_maint_in_region = ((maint_in_region * (type1_mult[inputs["Geography"][i]][maint_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type1_warr_in_region = ((warr_in_region * (type1_mult[inputs["Geography"][i]][warr_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])

    type2_admin_in_region = ((admin_in_region * (type2_mult[inputs["Geography"][i]][admin_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type2_maint_in_region = ((maint_in_region * (type2_mult[inputs["Geography"][i]][maint_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])
    type2_warr_in_region = ((warr_in_region * (type2_mult[inputs["Geography"][i]][warr_row_num])) / 1000000) * (inputs["Number of stations"][i] / inputs["Number of years for analysis"][i])

    avg_kwh = 0
    if inputs["[Average] amount of kWh dispensed per charge session"][i] in kWh_dict.keys():
        avg_kwh = kWh_dict[inputs["[Average] amount of kWh dispensed per charge session"][i]]
    else:
        avg_kwh = inputs["[Average] amount of kWh dispensed per charge session"][i]

    type1_total_warr_in_region = (type1_admin_in_region + type1_maint_in_region + type1_warr_in_region) * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    
    if str(inputs["yes/no to include induced effects"][i]).strip().lower() == "yes":
        induced_total_warr_region = ((type2_admin_in_region + type2_maint_in_region + type2_warr_in_region) - (type1_admin_in_region + type1_maint_in_region + type1_warr_in_region)) * inputs["[Average] sessions per month"][i] * avg_kwh * 12
    else:
        induced_total_warr_region = 0
    
    
    warr_type1_sum = 0
    warr_induced_sum = 0
    for j in range((inputs["Number of years for analysis"][i])):
        warr_type1_sum += type1_total_warr_in_region * (j+1)
        warr_induced_sum += induced_total_warr_region * (j+1)
    
    warr_sector_total = warr_type1_sum + warr_induced_sum
    warr_sec_emp.append(warr_sector_total)
    warr_type1_sum_list.append(warr_type1_sum)
    warr_induced_sum_list.append(warr_induced_sum)

for i in range(len(warr_type1_sum_list)):
    curr_row = []
    curr_row.append(warr_type1_sum_list[i])
    curr_row.append(warr_induced_sum_list[i])
    curr_row.append(warr_sec_emp[i])
    warr_totals.append(curr_row)

print(warr_type1_sum_list)
print(warr_induced_sum_list)
print(warr_sec_emp)
print(warr_totals)



[0.11936418122827559, 0.12431699904244206, 0.12431698854859397, 0.3162789281734438, 0.3094838959585906, 0.10935180581801295, 0.10935180051099687, 0.2165999961840757, 0.05907272623202063]
[0.049897442685299255, 0, 0.10908312245890213, 0.20954228143882828, 0, 0.05129633115267009, 0.05129632870598205, 0.1437238327616996, 0.03919740893500897]
[0.16926162391357485, 0.12431699904244206, 0.23340011100749608, 0.5258212096122721, 0.3094838959585906, 0.16064813697068303, 0.1606481292169789, 0.3603238289457753, 0.09827013516702959]
[[0.11936418122827559, 0.049897442685299255, 0.16926162391357485], [0.12431699904244206, 0, 0.12431699904244206], [0.12431698854859397, 0.10908312245890213, 0.23340011100749608], [0.3162789281734438, 0.20954228143882828, 0.5258212096122721], [0.3094838959585906, 0, 0.3094838959585906], [0.10935180581801295, 0.05129633115267009, 0.16064813697068303], [0.10935180051099687, 0.05129632870598205, 0.1606481292169789], [0.2165999961840757, 0.1437238327616996, 0.36032382894577

##### Creating Dataframe with total output values

In [109]:
warr_sect_df = pd.DataFrame(warr_totals, index = row_headings_ops, columns = column_total_headings_ops)
warr_sect_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,0.119364,0.049897,0.169262
RUN 2,0.124317,0.000000,0.124317
RUN 3,0.124317,0.109083,0.233400
RUN 4,0.316279,0.209542,0.525821
RUN 5,0.309484,0.000000,0.309484
RUN 6,0.109352,0.051296,0.160648
RUN 7,0.109352,0.051296,0.160648
RUN 8,0.216600,0.143724,0.360324
RUN 9,0.059073,0.039197,0.098270


#### Station Operations Employment Table

##### Performing calculations for Stations Operations Employment Table

In [110]:
# Calculating totals for all EVSE components:

stat_ops_type1 = []
stat_ops_type2 = []
stat_ops_induced = []
station_ops_totals = []

for i in range(len(inputs)):
    type1 = (elec_type1_sum_list[i] + retail_type1_sum_list[i] + ad_type1_sum_list[i] + data_type1_sum_list[i] + warr_type1_sum_list[i])
    induced = (elec_induced_sum_list[i] + retail_induced_sum_list[i] + ad_induced_sum_list[i] + data_induced_sum_list[i] + warr_induced_sum_list[i]) 
    type2 = (elec_sec_emp[i] + retail_sect_emp[i] + ad_emp[i] + data_sec_emp[i] + warr_sec_emp[i])
    stat_ops_type1.append(type1)
    stat_ops_induced.append(induced)
    stat_ops_type2.append(type2)

for i in range(len(stat_ops_type1)):
    curr_row = []
    curr_row.append(stat_ops_type1[i])
    curr_row.append(stat_ops_induced[i])
    curr_row.append(stat_ops_type2[i])
    station_ops_totals.append(curr_row)

station_ops_totals

[[0.9048015295006395, 0.45668599613090266, 1.3614875256315422],
 [0.8195712986228465, 0, 0.8195712986228465],
 [1.2757238440880991, 1.2779216368101538, 2.553645480898253],
 [2.6333009904896607, 1.9338038793051395, 4.5671048697948],
 [0.8632531261880472, 0, 0.8632531261880472],
 [0.3341842526786941, 0.16583522923463695, 0.500019481913331],
 [0.2625381703208923, 0.1277977478827322, 0.3903359182036245],
 [20.71674982494794, 8.332903975425873, 29.04965380037381],
 [5.650022679531256, 2.2726101751161476, 7.922632854647404]]

##### Creating Dataframe with total output values

In [111]:
statops_df = pd.DataFrame(station_ops_totals, index = row_headings_ops, columns = column_total_headings_ops)
statops_df

,Supply Chain Employment,Induced Employment,Total Employment
RUN 1,0.904802,0.456686,1.361488
RUN 2,0.819571,0.000000,0.819571
RUN 3,1.275724,1.277922,2.553645
RUN 4,2.633301,1.933804,4.567105
RUN 5,0.863253,0.000000,0.863253
RUN 6,0.334184,0.165835,0.500019
RUN 7,0.262538,0.127798,0.390336
RUN 8,20.716750,8.332904,29.049654
RUN 9,5.650023,2.272610,7.922633


## Writing Output to Final Sheet

Writing output and resulting calculations to input excel sheet

##### Station Development Employment:

In [112]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    statdev_df.to_excel(writer, sheet_name = "Station Development")

# Validation - Check to see if output matches expected values

##### Station Operations Employment:

In [113]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    statops_df.to_excel(writer, sheet_name = "Station Operations")

# Validation - Check to see if output matches expected values

#### Civil Construction Employment:

In [114]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    civil_const_df.to_excel(writer, sheet_name = "Civil Construction")

# Validation - Check to see if output matches expected values

#### Electrical Construction Employment

In [115]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    elec_const_df.to_excel(writer, sheet_name = "Electrical Construction")

# Validation - Check to see if output matches expected values

#### Electricity Sector Employment		

In [116]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    elec_sect_df.to_excel(writer, sheet_name = "Electricity Sector")

# Validation - Check to see if output matches expected values

#### Retail Sector Employment		

In [117]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    retail_sect_df.to_excel(writer, sheet_name = "Retail Sector")

# Validation - Check to see if output matches expected values

#### Advertising Sector Employment	

In [118]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    ad_sect_df.to_excel(writer, sheet_name = "Advertising Sector")

# Validation - Check to see if output matches expected values

#### Data and Networking Sector Employment	

In [119]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    data_sect_df.to_excel(writer, sheet_name = "Data and Networking Sector")

# Validation - Check to see if output matches expected values

#### Warranty, Maintenance, Administrative Costs, & Access Fees Sector Employment														

In [120]:
# Final Part: Adding DataFrame as a new excel sheet in User-Input Excel sheet:
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:  
    warr_sect_df.to_excel(writer, sheet_name = "Warranty, Maintenance, Administrative Costs, & Access Fees Sector Employment Sector")

# Validation - Check to see if output matches expected values

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
